In [1]:
# Heat budget terms for the different layers
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

/Users/weiyangbao/miniconda3/envs/py37/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
Cp = 4220  # Specific heat content J/kg/C
rho = 1025 # Density
T0 = 9 # Initial temperature
ny = 9 # Number of grids in Y direction
ga = np.repeat(np.array([[.5],[1],[1.5],[4],[8]]),10) * 200 # Grid area
X = 50 # Number of cells in x direction

Area = np.empty([50, 9])
Area[:10, :] = 100
Area[10:20, :] = 200
Area[20:30, :] = 300
Area[30:40, :] = 800
Area[40:, :] = 1600


In [3]:
# Find the interface and vertical flux of theta
def Layer(u):
    # Layer interface, zero velocity crossing point
    z_th = np.empty(X)
    itf = np.empty(X, dtype=int)
    z_th[[0,-1]] = 0
    itf[[0,-1]] = 0
    for i in range(1,X-1):
        l = np.where(u[:, i] < 0)[-1][0]
        itf[i] = l
        z_th[i] = -w[l, i] * (pt[l-1:l+1,i].mean()-T0)
    return (itf, z_th)

In [13]:
files = sorted(glob.glob('/Users/weiyangbao/Documents/Model_outputs/State700/Hs*.nc'))
sd = np.array([10, 12, 15, 110, 150]) # Sill depth
f = 3 # Index of the left boundary
s = 28 # Index of the right boundary

wi = np.empty([50, s-f+1])
wi[:10, :] = 0.5 / 150
wi[10:20, :] = 1 / 150
wi[20:30, :] = 1.5 / 150
wi[30:40, :] = 4 / 150
wi[40:, :] = 8 / 150


def Volume(theta):
    tim = state.T.data
    dt = tim[1] - tim[0]
    Tvol1 = np.empty(len(tim))
    Tvol2 = np.empty(len(tim))
    Tvol = np.empty(len(tim))
    H1t = np.empty(len(tim))
    H2t = np.empty(len(tim))
    Ht = np.empty(len(tim))
    for t in range(len(tim)):
        thetaV = theta[t,:,f:s+1] * wi / (s-f+1)
        mask = np.ma.masked_equal(thetaV, 0)
        V = mask/mask * wi * 150 * 200 * 200 * ny
        V1x = np.empty(s-f+1)
        V2x = np.empty(s-f+1)
        T1x = np.empty(s-f+1)
        T2x = np.empty(s-f+1)
        for j in range(s-f+1):
            V1x[j] = V[:itf[f+j],j].sum()
            V2x[j] = V[itf[f+j]:,j].sum()
            T1x[j] = thetaV[:itf[f+j],j].sum()
            T2 = thetaV[itf[f+j]:,j]
            T2x[j] = T2[T2!=0].sum()
        
        Tvol1[t] = T1x.sum()
        Tvol2[t] = T2x.sum()
        Tvol[t] = thetaV[thetaV!=0].sum()
        V1 = V1x.sum()
        V2 = V2x.sum()
        
        
    H1t[0] = 0
    H2t[0] = 0
    Ht[0] = 0
    for i in range(1,len(tim)):
        H1t[i] = -(Tvol1[i]-Tvol1[i-1])/ dt * V1
        H2t[i] = -(Tvol2[i]-Tvol2[i-1])/ dt * V2
        Ht[i] = -(Tvol[i]-Tvol[i-1])/ dt * (V.sum())
        #H2t = np.diff(Tvol2).mean() / dt * V2
        #Ht = np.diff(Tvol).mean() / dt * (V.sum())
    H1 = H1t[-1] - H1t[1]
    H2 = H2t[-1] - H2t[1]
    H = Ht[-1] - Ht[1]
    return (H1, H2, H)


Hs = np.empty(len(sd)) # Heat flux through the ice front
Hs1 = np.empty(len(sd)) # 1: upper layer
Hx = np.empty(len(sd)) # Heat flux through the sill
Hx1 = np.empty(len(sd))
#Hx2 = np.empty(len(sd))
Hz = np.empty(len(sd)) # Vertical heat flux
H_stor1 = np.empty(len(sd))
H_stor2 = np.empty(len(sd))
H_stor = np.empty(len(sd))
for i in range(len(sd)):
    state0 = xr.open_dataset(files[i])
    t0 = 39 # Start from day 20
    tn = len(state0.T) # End day
    state = state0.isel(Y = range(1,10), T = range(t0,tn)) # Remove the boundary walls in y direction
    pt = state.Temp.data.mean(axis=(0,2)) # Z, X
    w = state.W.data.mean(axis=(0,2)) # Zl, X
    u0 = state.U.data # dims = T, Z, Y, Xp1
    u_x = 0.5 * (u0[:, :, :, 1:] + u0[:, :, :, :-1]) # Convert the value to grid center in x direction
    u = u_x.mean(axis=(0,2)) # dims = Z, X 
    itf, z_th = Layer(u)
    
    theta = state.Temp.data.mean(2) # T, Z, X (averaged in Y)


    H1, H2, H = Volume(theta)
    H_stor1[i] = H1 * Cp * rho 
    H_stor2[i] = H2 * Cp * rho
    H_stor[i] = H * Cp * rho
    Hz[i] = z_th[s:f+1].sum() * 200 * 200 * ny
    Hs[i] = np.sum(u[:,f] * (pt[:,f]-T0) * ga) * ny
    Hs1[i] = np.sum(u[:itf[f],f] * (pt[:itf[f],f]-T0) * ga[:itf[f]]) * ny
    Hx[i] = -np.sum(u[:,s] * (pt[:,s]-T0) * ga) * ny
    Hx1[i] = -np.sum(u[:itf[s],s] * (pt[:itf[s],s]-T0) * ga[:itf[s]]) * ny
    #Hx2[i] = -np.sum(u[itf[r]:,r] * (pt[itf[r]:,r]-T0) * ga[itf[r]:]) * ny


In [11]:
H_stor1, H_stor2, H_stor

(array([ 3734364.03575425,  2192357.56030484, -4577263.34475579,
          307675.29234683, -2867704.34308612]),
 array([1.18903415e+09, 5.64882830e+08, 9.02259404e+07, 1.73088577e+07,
        8.05082536e+06]),
 array([ 1.32977736e+09,  6.38169858e+08,  3.45512655e+07,  2.28047319e+07,
        -2.97331735e+07]))

In [24]:
"{:.2E}".format(H_stor1[0]), "{:.2E}".format(H_stor2[2]), "{:.2E}".format(H_stor1[3]), "{:.2E}".format(H_stor1[4])

('-3.73E+06', '-9.02E+07', '-3.08E+05', '2.87E+06')

In [8]:
dt

NameError: name 'dt' is not defined